# Scaling Fmax from muscle volumes

In [1]:
import opensim as osim
import numpy as np
import re
import os
import tkinter as tk
from tkinter import filedialog

from muscle_volume_calculator import muscle_volume_calculator

## Required information

To scale Fmax from muscle volumes, we require the height (m) and mass (kg) of the subject.

In [2]:
subject_height = float(input('Enter subjects height (m): '))
subject_mass = float(input('Enter subjects mass (kg): '))

root = tk.Tk()
root.withdraw()

subject_path = filedialog.askopenfilename(initialdir = os.getcwd(),title = "Select subject model",filetypes = (("osim files","*.osim"),("all files","*.*")))

Enter subjects height (m): 1.706
Enter subjects mass (kg): 76.2


## Muscle volumes
Handsfield (2014) found a linear relationship between $muscle~volume$ and ($height \times mass$) for the lower limbs.

This code should only be used for lower limb muscles.

In [3]:
# Calculate muscle volumes using Handsfield (2004)
osim_abbr, muscle_volume = muscle_volume_calculator(subject_height, subject_mass)

In [4]:
# Load OpenSim model and its muscle set
osim_model = osim.Model(subject_path)
all_muscles = osim_model.getMuscles()

all_muscles_names = [None] * all_muscles.getSize()
old_value = np.zeros((all_muscles.getSize(), 1))
optimal_fibre_length = np.zeros((all_muscles.getSize(), 1))
pen_ang_at_opt_fib_length = np.zeros((all_muscles.getSize(), 1))

for i in range(all_muscles.getSize()):
    all_muscles_names[i] = all_muscles.get(i).getName()
    old_value[i, 0] = all_muscles.get(i).getMaxIsometricForce()
    optimal_fibre_length[i, 0] = all_muscles.get(i).getOptimalFiberLength()
    pen_ang_at_opt_fib_length[i, 0] = all_muscles.get(i).getPennationAngleAtOptimalFiberLength()

# Convert optimal fiber length from m to cm to match volume units (cm^3)
optimal_fibre_length = optimal_fibre_length * 100

all_muscles_names_cut = [None] * all_muscles.getSize()

for i in range(all_muscles.getSize()):
    # Delete trailing _r or _l
    curr_mus_name = all_muscles_names[i][:-2]

    # Split the name from any digit in its name and only keep the first string
    all_muscles_names_cut[i] = re.split(r'\d', curr_mus_name)[0]

# Calculate ratio of old max isometric forces for multiple-lines-of-action muscles
new_abs_volume = np.zeros((all_muscles.getSize(), 1))
frac_of_group = np.zeros((all_muscles.getSize(), 1))

In [5]:
for i in range(all_muscles.getSize()):
    curr_mus_name = all_muscles_names_cut[i]

    try: 
        curr_index = osim_abbr.index(curr_mus_name)
        curr_value = muscle_volume[curr_index]
        new_abs_volume[i,0] = curr_value

        curr_muscle_name_index = []
        tmp_index = [j for j in range(len(all_muscles_names_cut)) if all_muscles_names_cut[j] == curr_mus_name]
        curr_muscle_name_index.append(tmp_index[:int(len(tmp_index)/2)])

    except ValueError: # muscle name not found
        # The peroneus longus/brevis and the extensors (EDL, EHL) have to be treated separately as they are
        # represented as a combined muscle group in Handsfield (2014).
        if ('per_brev' in curr_mus_name) or ('per_long' in curr_mus_name):
            curr_muscle_name_index = []

            tmp_index = all_muscles_names_cut.index('per_brev') # .index() finds the first occuring element
            curr_muscle_name_index.append(tmp_index)

            tmp_index = all_muscles_names_cut.index('per_long') # .index() finds the first occuring element
            curr_muscle_name_index.append(tmp_index)

            curr_index = osim_abbr.index('per_')
            curr_value = muscle_volume[curr_index]
            new_abs_volume[i] = curr_value

        elif ('ext_dig' in curr_mus_name) or ('ext_hal' in curr_mus_name):
            curr_muscle_name_index = []

            tmp_index = all_muscles_names_cut.index('ext_dig') # .index() finds the first occuring element
            curr_muscle_name_index.append(tmp_index)

            tmp_index = all_muscles_names_cut.index('ext_hal') # .index() finds the first occuring element
            curr_muscle_name_index.append(tmp_index)

            curr_index = osim_abbr.index('ext_')
            curr_value = muscle_volume[curr_index]
            new_abs_volume[i] = curr_value

        else:
            curr_muscle_name_index = []
            tmp_index = [j for j in range(len(all_muscles_names_cut)) if all_muscles_names_cut[j] == curr_mus_name]
            curr_muscle_name_index.append(tmp_index[:int(len(tmp_index)/2)])

    frac_of_group[i,0] = old_value[i,0]/np.sum(old_value[np.array(tuple(curr_muscle_name_index))])

## Calculate the new maximal isometric muscle forces

In [6]:
specific_tension = 61 # N/cm^2 from Zajac 1989
new_volume = frac_of_group*new_abs_volume

# Maximum isometric muscle force
max_iso_muscle_force = specific_tension * (new_volume/optimal_fibre_length) * np.cos(pen_ang_at_opt_fib_length)

# Maximum isometric fibre force
max_iso_fibre_force = specific_tension * (new_volume/optimal_fibre_length)

## Update muscles of loaded model

In [7]:
for i in range(all_muscles.getSize()):
    # Only update if new value is not 0. Else do not override the original value
    if max_iso_muscle_force[i][0] != 0:
        all_muscles.get(i).setMaxIsometricForce(max_iso_fibre_force[i][0])

# Create and set new model name by adding '_newFmax' at the end
osim_name_old = osim_model.getName().split('.')[0]
osim_name_new = osim_name_old + '_newFmax.osim'
osim_model.setName(osim_name_new)

print_new_model = 'yes'

# Print new model in specified path
if print_new_model == 'yes':
    print_path = subject_path.split('.')[0] + '_newFmax.osim'

    osim_model.printToXML(print_path)

else:
    print('----------------------------------------')
    print('WARNING: no new model file was printed!')
    print('Change in settings and re-run if wanted.')
    print('----------------------------------------')